In [38]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter

import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt

import lsqfit

from tqdm import tqdm

from scipy import linalg as la

import copy
import corrfitter as cf

In [39]:
BINSIZE  = {
    'MediumCoarse':13,
    'Coarse-2':    16,
    'Coarse-1':    11,
    'Coarse-Phys': 19,
    'Fine-1':      16,
    'Fine-Phys':   16,
    'SuperFine':   22
}

In [40]:
# Correlator specifics
ENSEMBLE = 'Coarse-1'
MESON    = 'Dst'
MOMENTUM = '300'

DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'

SMSLIST  = ['1S-1S','d-d','d-1S']

# Specifics of the analysis
TRANGE_EFF = (10,18) 
TRANGE     = (4,18)
NSTATES    = 3
JKFIT      = False

In [41]:
io   = CorrelatorIO(ENSEMBLE,MESON,MOMENTUM,PathToDataDir=DATA_DIR)
stag = StagFitter(
    io       = io,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = SMSLIST
)
cov_specs = dict(scale=True, shrink=True, cutsvd=1e-12)

In [125]:
effm,effa = stag.meff(TRANGE_EFF,**cov_specs)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


# eigen basis by hand (whole matrix)

In [166]:
xdata,y,ysample = stag.format(alljk=True,**cov_specs)

t0 = 1
t1 = 3

ebasis = np.array([
    [ysample['1S-1S','Bot'],ysample['d-1S','Bot']],
    [ysample['d-1S' ,'Bot'],ysample['d-d','Bot']]
])
ebasis = np.moveaxis(ebasis,[0,1],[-2,-1])

Ct = ebasis.mean(axis=0)
eval, evec = la.eigh(Ct[t1],b=Ct[t0])

c00 = []
c11 = []
c01 = []
c10 = []
for jk in tqdm(range(ebasis.shape[0])):
    _00,_11,_10,_01 = [],[],[],[]
    for t in range(Ct.shape[0]):
        _00.append( evec[:,0] @ ebasis[jk,t] @ evec[:,0] ) 
        _11.append( evec[:,1] @ ebasis[jk,t] @ evec[:,1] ) 
        _01.append( evec[:,0] @ ebasis[jk,t] @ evec[:,1] ) 
        _10.append( evec[:,1] @ ebasis[jk,t] @ evec[:,0] ) 
    c00.append(_00)
    c11.append(_11)
    c01.append(_01)
    c10.append(_10)

100%|██████████| 90/90 [00:00<00:00, 2895.37it/s]


In [169]:
tmp = np.hstack([c00,c11,c01,c10])
c1,c2,c3,c4 = gv.gvar(
    tmp.mean(axis=0),
    np.cov(tmp,rowvar=False) * (tmp.shape[0]-1)
).reshape((4,tmp.shape[1]//4))

ydata = {'c00':c1, 'c11':c2, 'c01':c3, 'c10':c4}

In [ ]:
models = []
for tag in ydata:
    fcn = cf.Corr2(
        datatag = tag,
        tp      = stag.Nt,
        s       = (1.,-1.),
        tdata   = xdata,
        tmin    = 4,
        tmax    = 18,
        dE      = ('dE','dE.o'),
        a       = (f'Z_{tag}',f'Z_{tag}.o'),
        b       = (f'Z_{tag}',f'Z_{tag}.o')
    )
    models.append(fcn)

# eigen basis by hand 2

In [171]:
xdata,y,ysample = stag.format(alljk=True,**cov_specs)

In [172]:
t0 = 1
t1 = 3

In [173]:
ebasis = np.array([
    [ysample['1S-1S','Bot'],ysample['d-1S','Bot']],
    [ysample['d-1S' ,'Bot'],ysample['d-d','Bot']]
])
ebasis = np.moveaxis(ebasis,[0,1],[-2,-1])

In [174]:
Ct = ebasis.mean(axis=0)
eval, evec = la.eigh(Ct[t1],b=Ct[t0])

Ctmax = []
Ctmin = []
for jk in tqdm(range(ebasis.shape[0])):
    cmax,cmin = [],[]
    for t in range(Ct.shape[0]):
        cmin.append( evec[:,0] @ ebasis[jk,t] @ evec[:,0] ) 
        cmax.append( evec[:,1] @ ebasis[jk,t] @ evec[:,1] ) 
    Ctmax.append(cmax)
    Ctmin.append(cmin)

100%|██████████| 90/90 [00:00<00:00, 5713.19it/s]


In [175]:
tmp = np.hstack([Ctmax,Ctmin])

c1,c2 = gv.gvar(
    tmp.mean(axis=0),
    np.cov(tmp,rowvar=False) * (tmp.shape[0]-1)
).reshape((2,tmp.shape[1]//2))

ydata = {'cmax':c1, 'cmin':c2}

In [193]:
ks = {'cmax':'cmin','cmin':'cmax'}

models = []
for tag in ydata:
    fcn = cf.Corr2(
        datatag = tag,
        tp      = stag.Nt,
        s       = (1.,-1.),
        tdata   = xdata,
        tmin    = 4,
        tmax    = 18,
        dE      = ('dE','dE.o'),
        a       = (f'Z_{tag}',f'Z_{tag}.o'),
        b       = (f'Z_{ks[tag]}',f'Z_{ks[tag]}.o')
    )
    models.append(fcn)

In [194]:
NSTATES = 1
priors = {
    'log(dE)'      : gv.log(gv.gvar(NSTATES * ['0.5(5)'])),
    'log(dE.o)'    : gv.log(gv.gvar(NSTATES * ['0.5(5)'])),
    'log(Z_cmax)'  : gv.log(gv.gvar(NSTATES * ['1(1)'])),
    'log(Z_cmin)'  : gv.log(gv.gvar(NSTATES * ['1(1)'])),
    'log(Z_cmax.o)': gv.log(gv.gvar(NSTATES * ['1(1)'])),
    'log(Z_cmin.o)': gv.log(gv.gvar(NSTATES * ['1(1)'])),
}
priors['log(dE)'][0] = gv.log(gv.gvar(effm.mean,0.1))

In [195]:
fitter = cf.CorrFitter(models=models)
fit = fitter.lsqfit(data=ydata, prior=priors)

In [196]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 8.1e+03 [30]    Q = 0    logGBF = -1.2046e+05

Parameters:
  log(Z_cmax) 0     -0.06 (71)      [    0 ± 1.0 ]  
  log(Z_cmin) 0     -0.06 (71)      [    0 ± 1.0 ]  
      log(dE) 0    0.6990 (43)      [ 0.186 (83) ]  *****
log(Z_cmax.o) 0    -11.70 (79)      [    0 ± 1.0 ]  *****
log(Z_cmin.o) 0    -11.70 (79)      [    0 ± 1.0 ]  *****
    log(dE.o) 0     -3.43 (86)      [ -0.7 (1.0) ]  **
--------------------------------------------------
       Z_cmax 0      0.94 (67)      [  1.0 (1.0) ]  
       Z_cmin 0      0.94 (67)      [  1.0 (1.0) ]  
           dE 0    2.0117 (86)      [  1.20 (10) ]  *****
     Z_cmax.o 0   8.3(6.6)e-06      [  1.0 (1.0) ]  
     Z_cmin.o 0   8.3(6.6)e-06      [  1.0 (1.0) ]  
         dE.o 0     0.032 (28)      [  0.50 (50) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 32/0.0)
  fitter = scipy_least_squares    method = trf



# Disentangled fit states

In [416]:
def PeriodicExpDecay_jax(Nt):
    return lambda t,E,Z: Z * ( np.exp(-E*t) + np.exp(-E*(Nt-t)) ) 

def _model(Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        # E0, E1 = p['E'][0], p['E'][0] + np.exp(p['E'][1])
        E0, E1 = p['E'][0], p['E'][1]
        # Z0 = (np.exp(p[f'Z_{sm1}_{pol}'][0]) * np.exp(p[f'Z_{sm2}_{pol}'][0]))/2/E0
        # Z1 = (np.exp(p[f'Z_{sm1}_{pol}'][1]) * np.exp(p[f'Z_{sm2}_{pol}'][1]))/2/E1
        Z0 = (p[f'Z_{sm1}_{pol}'][0] * p[f'Z_{sm2}_{pol}'][0])/2/E0
        Z1 = (p[f'Z_{sm1}_{pol}'][1] * p[f'Z_{sm2}_{pol}'][1])/2/E1

        ans = PeriodicExpDecay_jax(Nt)(t,E0,Z0) + PeriodicExpDecay_jax(Nt)(t,E1,Z1) * (-1)**(t+1)

        Es = [E0,E1]
        for i in range(2,2*Nstates):
            # Ei = Es[i-2] + np.exp(p['E'][i])
            Ei = Es[i-2] + p['E'][i]
            Z = p[f'Z_{sm if mix else sm1}_{pol}'][i-2 if mix else i]**2
            ans += PeriodicExpDecay_jax(Nt)(t,Ei,Z) * (-1)**(i*(t+1))

            Es.append(Ei)
        return ans

    return aux

In [419]:
effm,effa = stag.meff(TRANGE_EFF,**cov_specs)

pr = stag.priors(NSTATES,Meff=effm,Aeff=effa)
# for n in [0,1]:
#     e = pr['E'][0]
#     if n==1:
#         e += np.exp(pr['E'][1])

#     for sm in ['1S','d']:
#         for pol in ['Par','Bot']:
#             z = np.log(effa[f'{sm}-{sm}',pol] * 2 * e)
#             pr[f'Z_{sm}_{pol}'][n] = gv.gvar(z.mean,z.sdev*20)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


In [428]:
pr['E'][1:] = gv.log(np.exp(pr['E'][1:]))

pr['Z_1S_Bot'][:2] = gv.log(np.exp(pr['Z_1S_Bot'][:2]))
pr['Z_1S_Par'][:2] = gv.log(np.exp(pr['Z_1S_Par'][:2]))
pr['Z_d_Bot'][:2]  = gv.log(np.exp(pr['Z_d_Bot'][:2]))
pr['Z_d_Par'][:2]  = gv.log(np.exp(pr['Z_d_Par'][:2]))

In [430]:
xdata, ydata = stag.format(TRANGE, flatten=True, **cov_specs)

def model(xdata,pdict):
    return np.concatenate([
        _model(NSTATES,stag.Nt,smr,pol)(xdata,pdict) for smr,pol in stag.keys
    ])  

In [436]:
fit = lsqfit.nonlinear_fit(
    data   = (xdata,ydata),
    fcn    = model,
    prior  = pr,
)

print(fit)

/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:118: RuntimeWarning: overflow encountered in multiply
  suf = s * uf
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:141: RuntimeWarning: invalid value encountered in scalar multiply
  alpha = max(0.001 * alpha_upper, (alpha_lower * alpha_upper)**0.5)
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:112: RuntimeWarning: overflow encountered in square
  denom = s**2 + alpha
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:113: RuntimeWarning: invalid value encountered in divide
  p_norm = norm(suf / denom)
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:115: RuntimeWarning: invalid value encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
/Users/pietro/opt/anaconda3/envs/stat/lib

Least Square Fit:
  chi2/dof [dof] = inf [90]    Q = 0    logGBF = -inf

Parameters:
            E 0      1.20432 (0)       [   1.204 (48) ]  
              1     -1.81356 (0)       [   -1.81 (58) ]  
              2     -1.08762 (0)       [   -1.09 (80) ]  
              3     -2.35255 (0)       [   -2.4 (2.5) ]  
              4     -2.35255 (0)       [   -2.4 (2.5) ]  
              5     -2.35255 (0)       [   -2.4 (2.5) ]  
     Z_1S_Bot 0   -0.0955977 (0)       [ -0.10 (1.00) ]  
              1         -1.2 (0)       [   -1.2 (1.2) ]  
              2          0.5 (0)       [    0.5 (1.5) ]  
              3          0.5 (0)       [    0.5 (1.5) ]  
              4          0.5 (0)       [    0.5 (3.0) ]  
              5          0.5 (0)       [    0.5 (3.0) ]  
     Z_1S_Par 0    0.0645125 (0)       [  0.06 (1.00) ]  
              1         -1.2 (0)       [   -1.2 (1.2) ]  
              2          0.5 (0)       [    0.5 (1.5) ]  
              3          0.5 (0)       [    0

/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/lsqfit/_scipy.py:175: RuntimeWarning: overflow encountered in square
  self.cov = numpy.dot(_VT.T / _s**2, _VT)
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/lsqfit/__init__.py:729: RuntimeWarning: overflow encountered in square
  self.chi2 = numpy.sum(fit.f**2)


In [432]:
print( np.exp(fit.p['Z_1S_Bot'][0]))
print( np.exp(fit.p['Z_d_Bot'][0] ))

0.90883(0)
0.134519(0)


# Eigen basis by hand

In [328]:
ebasis = np.array([
    [ysample['1S-1S','Bot'],ysample['d-1S','Bot']],
    [ysample['d-1S' ,'Bot'],ysample['d-d','Bot']]
])
ebasis = np.moveaxis(ebasis,[0,1],[-2,-1])

C_t = ebasis.mean(axis=0)

In [280]:
t0,t1 = 9, 18
l,v = la.eigh(C_t[t1],b=C_t[t0])

eC_t = []
for t in range(C_t.shape[0]):
    eC_t.append([
        [v[0]@C_t[t]@v[0], v[0]@C_t[t]@v[1]], 
        [v[1]@C_t[t]@v[0], v[1]@C_t[t]@v[1]]
    ]) 
eC_t = np.array(eC_t)

In [322]:
eval, evec = la.eigh(C_t[t1],b=C_t[t0])
print(eval,evec)

[1.08357916e-05 6.35224702e-05] [[ -94.35001733 -659.4854221 ]
 [-790.91806825 4663.86977389]]


# Eigen basis

In [189]:
xdata,ydata = stag.format(scale=True,shrink=True)

ydict = {}
for sm,pol in ydata:
    sm1,sm2 = sm.split('-')
    k = f'{sm1}-{sm2}_{pol}'
    ydict[k] = ydata[sm,pol]


for pol in stag.data.polarization.values:
    ydict[f'1S-d_{pol}'] = copy.deepcopy(ydict[f'd-1S_{pol}'])



In [200]:
basis = cf.EigenBasis(
    ydict,
    keyfmt = '{s1}-{s2}_Bot',
    srcs   = ['1S','d'],
    t      = (11,17) ,
    tdata  = xdata,
    osc    = True
)
prior = basis.make_prior(nterm=2, keyfmt='m.{s1}')

In [201]:
prior

BufferDict({'m.1S': array([-0.19(13), -1.26(38)], dtype=object), 'm.d': array([-0.012(19), -0.189(57)], dtype=object), 'log(m.dE)': array([0.03(22), -1.5(1.0)], dtype=object), 'o.1S': array([-0.3(1.3), -0.3(1.3)], dtype=object), 'o.d': array([-0.04(19), -0.04(19)], dtype=object), 'log(o.dE)': array([0.03(22), -1.5(1.0)], dtype=object)})

In [172]:
effm,effa = stag.meff(TRANGE_EFF,**cov_specs)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


In [174]:
effa

BufferDict({('1S-1S', 'Bot'): 0.826(30), ('1S-1S', 'Par'): 1.138(35), ('d-1S', 'Bot'): 0.1191(25), ('d-1S', 'Par'): 0.1609(28), ('d-d', 'Bot'): 0.01810(30), ('d-d', 'Par'): 0.02353(34)})

# Classical fit

In [409]:
effm,effa = stag.meff(TRANGE_EFF,**cov_specs)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


In [410]:
priors = stag.priors(NSTATES,Meff=effm,Aeff=effa)

In [411]:
fit = stag.fit(
    Nstates = NSTATES,
    trange  = TRANGE,
    priors  = priors,
    # jkfit   = True,
    **cov_specs
)

In [414]:
print(np.exp(fit.p['Z_1S_Bot'][0]) * 2 * fit.p['E'][0])
print(np.exp(fit.p['Z_d_Bot'][0]) * 2 * fit.p['E'][0] )

1.69(29)
0.253(44)


In [213]:
fit['Z_1S_Bot'][:,0]

array([-0.26900711, -0.34088508, -0.29902848, -0.29014289, -0.2827814 ,
       -0.30701615, -0.28690166, -0.26892883, -0.27147336, -0.29028904,
       -0.26045125, -0.3033341 , -0.27198517, -0.29325767, -0.27802341,
       -0.27532271, -0.29873589, -0.29654462, -0.27473161, -0.2987464 ,
       -0.27360254, -0.30852834, -0.29267275, -0.29742953, -0.30713133,
       -0.29979372, -0.26494197, -0.27408599, -0.27646433, -0.30653436,
       -0.29504755, -0.27982753, -0.29115496, -0.29811514, -0.26293444,
       -0.26594842, -0.28827323, -0.26201076, -0.29109627, -0.28976959,
       -0.28301744, -0.25559455, -0.25780639, -0.28712346, -0.26927879,
       -0.29313795, -0.2721663 , -0.279077  , -0.30858129, -0.28196101,
       -0.2397884 , -0.25421923, -0.31897507, -0.25962441, -0.28062969,
       -0.26692793, -0.31892599, -0.2950269 , -0.27931767, -0.30968731,
       -0.28315379, -0.31007936, -0.31121948, -0.28510293, -0.29865923,
       -0.30813806, -0.28657545, -0.29245278, -0.30151635, -0.31

In [204]:
print(fitres['fit'].p['E'][0])
print(fitres['fit'].p['E'][1])

print('--------------------------------------')
print(np.exp(fitres['fit'].p['Z_1S_Bot'][0])**2)
print(np.exp(fitres['fit'].p['Z_1S_Bot'][1])**2)
print(np.exp(fitres['fit'].p['Z_1S_Bot'][2])**2)
print(np.exp(fitres['fit'].p['Z_1S_Bot'][3])**2)

print('--------------------------------------')
print(np.exp(fitres['fit'].p['Z_1S_Par'][0])**2)
print(np.exp(fitres['fit'].p['Z_1S_Par'][1])**2)
print(np.exp(fitres['fit'].p['Z_1S_Par'][2])**2)
print(np.exp(fitres['fit'].p['Z_1S_Par'][3])**2)

1.183(17)
-2.92(46)
--------------------------------------
0.51(16)
0.41(13)
6.4(1.5)
8.2(2.6)
--------------------------------------
0.70(23)
0.214(70)
9.6(2.4)
5.1(1.6)
